In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import linprog
from scipy.stats import spearmanr

from mhar import walk
import torch

np.random.seed(42)

In [2]:
class UTA:
    def __init__(self, df, device='cpu', n_probes=150):
        self.df = df
        self.device = device
        self.n_probes = n_probes
        self.better = []
        self.indifferent = []
        
    
    def addBetter(self, x, y):
        self.better.append([x, y])
        

    def getRanking(self):
        return self.__model("F")
        
        
    def __evaluate(self, mapping, x):
        result = np.zeros(self.df.shape[0])
        for i in range(len(mapping)):
            points = mapping[i]
            xp = []
            fp = []
            for q in points:
                xp += [q]
                fp += [x[points[q]]]
            xp = np.array(xp)
            fp = np.array(fp)
            fp = fp[np.argsort(xp)]
            xp = xp[np.argsort(xp)]
            result += np.interp(self.df[i], xp, fp)
        return result
    
    def __model(self, mode="F"):
        idx = list(set(sum(self.better, []) + sum(self.indifferent, [])))
        used = self.df.iloc[idx]
        mappings = []
        i = 0
        for c in self.df.columns:
            tmp = {}
            for q in sorted(used[c].values):
                if q not in tmp:
                    tmp[q] = i
                    i += 1
            mappings.append(tmp)
        c = np.zeros(i+1)
        a = np.zeros((len(self.better) + i - len(self.df.columns), len(c)))
        b = np.zeros(len(self.better) + i - len(self.df.columns))
        a_eq = np.zeros((2, len(c)))
        b_eq = np.zeros(2)

        for x in mappings:
            a_eq[0,min(x.values())] = 1
            a_eq[1,max(x.values())] = 1
        b_eq[1] = 1
        for i,(bet,wor) in enumerate(self.better):
            for j,q in enumerate(self.df.iloc[bet]):
                a[i][mappings[j][q]] = -1
            for j,q in enumerate(self.df.iloc[wor]):
                a[i][mappings[j][q]] = 1
        a[:len(self.better),-1] = 1
        i += 1
        bounds = [(0,1) for x in c]
        bounds[-1] = (1e-8,None)
        
        for x in mappings:
            k = list(sorted(x.values()))
            for j in range(len(k)-1):
                a[i][k[j]] = 1
                a[i][k[j+1]] = -1
                i += 1
                
        if mode == "F":
            c[-1] = -1
            return self.__evaluate(mappings, linprog(c, a, b, a_eq, b_eq, bounds=bounds).x)
        else:

            result = []
            am = torch.as_tensor(a)
            am = am.type(torch.FloatTensor)

            bm = torch.as_tensor(b.reshape(-1,1))
            bm = bm.type(torch.FloatTensor)

            aem = torch.as_tensor(a_eq)
            aem = aem.type(torch.FloatTensor)

            bem = torch.as_tensor(b_eq)
            bem = aem.type(torch.FloatTensor)

            x_0 = linprog(c, a, b, a_eq, b_eq, bounds=bounds).x
            x_0[-1] /= 2 
            x_0 = torch.as_tensor(x_0.reshape(-1,1))
            x_0 = x_0.type(torch.FloatTensor)
            X = walk(z=self.n_probes, 
             ai=am, 
             bi=bm, 
             ae = aem,
             be = bem,
             x_0=x_0, 
             T=1, 
             device=self.device, 
             warm=1, 
             seed=44, 
             thinning=10 
             ).numpy()
            for i in range(self.n_probes):
                result.append(self.__evaluate(mappings, X[i]))
            
            return spearmanr(result, axis=1)[0].min()
    
    def scorePair(self, x, y):
        self.better.append([x, y])
        result = self.__model("T")
        self.better = self.better[:-1] + [[y, x]]
        result = min(result, self.__model("T"))
        self.better = self.better[:-1]
        return result

In [3]:
df = pd.DataFrame(np.random.randn(50, 6))
df

,0,1,2,3,4,5
0,0.496714,-0.138264,0.647689,1.523030,-0.234153,-0.234137
1,1.579213,0.767435,-0.469474,0.542560,-0.463418,-0.465730
2,0.241962,-1.913280,-1.724918,-0.562288,-1.012831,0.314247
3,-0.908024,-1.412304,1.465649,-0.225776,0.067528,-1.424748
4,-0.544383,0.110923,-1.150994,0.375698,-0.600639,-0.291694
5,-0.601707,1.852278,-0.013497,-1.057711,0.822545,-1.220844
6,0.208864,-1.959670,-1.328186,0.196861,0.738467,0.171368
7,-0.115648,-0.301104,-1.478522,-0.719844,-0.460639,1.057122
8,0.343618,-1.763040,0.324084,-0.385082,-0.676922,0.611676
9,1.031000,0.931280,-0.839218,-0.309212,0.331263,0.975545


In [4]:
model = UTA(df.copy())

In [5]:
df.iloc[[49, 2]]

,0,1,2,3,4,5
49,0.357015,-0.69291,0.899600,0.307300,0.812862,0.629629
2,0.241962,-1.91328,-1.724918,-0.562288,-1.012831,0.314247


Element 49 is better on all criteria than element 2, thus scoring them would not improve model stability. The worse correlation between rankings returned by different models based on this comparison is close to -1

In [6]:
model.scorePair(49, 2)

Max non zero error:  tensor(0.)
Max non zero error:  tensor(0.)


-0.9892436974789915

In [7]:
df.iloc[[17, 49]]

,0,1,2,3,4,5
17,-0.342715,-0.802277,-0.161286,0.404051,1.886186,0.174578
49,0.357015,-0.692910,0.899600,0.307300,0.812862,0.629629


Element 49 is better than 17 on 4 out of 6 criteria and worse on the remaining 2. Scoring these two models would enrich our model with some preference information; however, it can still be better.

In [8]:
model.scorePair(17, 49)

Max non zero error:  tensor(0.)
Max non zero error:  tensor(0.)


-0.03539695499735843

In [9]:
df.iloc[[13, 6]]

,0,1,2,3,4,5
13,0.091761,-1.987569,-0.219672,0.357113,1.477894,-0.518270
6,0.208864,-1.959670,-1.328186,0.196861,0.738467,0.171368


Element 13 is better than element 6 on 3 criteria and worse on 3 remaining ones. Knowledge about the relation between these two alternatives can significantly improve model stability

In [10]:
model.scorePair(13, 6)

Max non zero error:  tensor(0.)
Max non zero error:  tensor(0.)


0.26570466570466567

We provide preference information $e_6 \succ e_{13}$ and then present ranking based on this one comparison

In [11]:
model.addBetter(6, 13)
df["score"] = model.getRanking()
df.sort_values("score", ascending=False)

,0,1,2,3,4,5,score
27,1.158596,-0.820682,0.963376,0.412781,0.822060,1.896793,1.000000e+00
29,0.276691,0.827183,0.013002,1.453534,-0.264657,2.720169,1.000000e+00
35,0.570891,1.135566,0.954002,0.651391,-0.315269,0.758969,1.000000e+00
23,0.813517,-1.230864,0.227460,1.307143,-1.607483,0.184634,1.000000e+00
49,0.357015,-0.692910,0.899600,0.307300,0.812862,0.629629,1.000000e+00
30,0.625667,-0.857158,-1.070892,0.482472,-0.223463,0.714000,1.000000e+00
26,1.865775,0.473833,-1.191303,0.656554,-0.974682,0.787085,1.000000e+00
8,0.343618,-1.763040,0.324084,-0.385082,-0.676922,0.611676,1.000000e+00
20,0.791032,-0.909387,1.402794,-1.401851,0.586857,2.190456,1.000000e+00
45,1.441273,-1.435862,1.163164,0.010233,-0.981509,0.462103,1.000000e+00


In [12]:
df.iloc[[17, 49]]

,0,1,2,3,4,5,score
17,-0.342715,-0.802277,-0.161286,0.404051,1.886186,0.174578,0.666667
49,0.357015,-0.692910,0.899600,0.307300,0.812862,0.629629,1.000000


Now the comparison between 17 and 49 is a nice addition to the model since it provides information that was not available in the first comparison.

In [13]:
model.scorePair(17, 49)

Max non zero error:  tensor(0.)
Max non zero error:  tensor(0.)


0.7730612244897959

We provide second comparison $e_{17} \succ e_{49}$ and present ranking based on the updated model

In [14]:
model.addBetter(17, 49)
df["score"] = model.getRanking()
df.sort_values("score", ascending=False)

,0,1,2,3,4,5,score
17,-0.342715,-0.802277,-0.161286,0.404051,1.886186,0.174578,1.000000e+00
33,0.058209,-1.142970,0.357787,0.560785,1.083051,1.053802,7.500000e-01
23,0.813517,-1.230864,0.227460,1.307143,-1.607483,0.184634,7.500000e-01
34,-1.377669,-0.937825,0.515035,0.513786,0.515048,3.852731,7.500000e-01
27,1.158596,-0.820682,0.963376,0.412781,0.822060,1.896793,7.500000e-01
29,0.276691,0.827183,0.013002,1.453534,-0.264657,2.720169,7.500000e-01
41,-0.474945,-0.653329,1.765454,0.404982,-1.260884,0.917862,7.500000e-01
35,0.570891,1.135566,0.954002,0.651391,-0.315269,0.758969,7.500000e-01
30,0.625667,-0.857158,-1.070892,0.482472,-0.223463,0.714000,7.500000e-01
26,1.865775,0.473833,-1.191303,0.656554,-0.974682,0.787085,7.500000e-01
